In [16]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re, time,csv
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [17]:
def get_links_from_page(writer:csv.writer):

    results=driver.find_elements(by=By.CSS_SELECTOR,value='article[class="m-statement m-statement--is-medium m-statement--true"]')
    
    for result in results: # for each story div
        
        fact_date,platform,text,name,truthfullness_category='NA','NA','NA','NA','NA'
       
        try: # try to get the name
            name_box=result.find_element(by=By.CSS_SELECTOR,value='a[class="m-statement__name"]')
            name=name_box.text
            #print(name)
        except NoSuchElementException as e:
            print('could not extract name')
            
        try: 
            date_box=result.find_element(by=By.CSS_SELECTOR,value='div[class="m-statement__desc"]')
            fact_date=date_box.text
            #print(fact_date)

        except NoSuchElementException as e:
            print('could not extract date')
            
        try: 
            review_text_box=result.find_element(by=By.CSS_SELECTOR,value='div[class="m-statement__quote"]')
            text=review_text_box.text
            #print(text)

        except NoSuchElementException as e:
            print('could not extract text')
    
            
        try:
            truthfullness_box=result.find_element(by=By.CSS_SELECTOR,value='div[class="c-image"]')
            truthfullness_category=truthfullness_box.get_attribute('img.alt')
            print(truthfullness_category)

        except NoSuchElementException as e:
            print('could not extract truthfullness_category')
        
        
            
        #write a new row for this story
        writer.writerow([name,fact_date,text,truthfullness_category])

In [18]:
def scrape_query(query:str,
                delay:int = 2 # number of seconds to wait 
                ):

    #create a new csv writer for the story links
    fw=open('data.csv','w')
    writer=csv.writer(fw,lineterminator='\n')
    writer.writerow(['Date','Platform','Text','Name','Truthfulness Category'])

    url='https://www.politifact.com/factchecks/list/?category=truth-o-meter&ruling='+query # create the url
    print("URL")

    driver.get(url) # visit the page

    input() # wait until we can click on the cookie overlayright

    page_cnt=1 # keep track of page count

    while page_cnt<10: # keep going until there are no more pages

        print('page',page_cnt) # print current page count

        page_cnt+=1 # increment 
        
        #extract and write the links from the current page
        get_links_from_page(writer) 
        try:
            # wait until the next button appears
            next_button = WebDriverWait(driver, delay).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[class*="c-button c-button--hollow"]' )))
        
        except TimeoutException:
            break
    
            
        #click the next button and wait 
        #next_button.click()
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(delay)
  
    fw.close()

In [19]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [20]:
driver.maximize_window()

In [21]:
query='true'

In [22]:
scrape_query(query)

URL
page 1
Elizabeth Warren
stated on August 28, 2022 in an interview on CNN:
"40% of the folks who have student loans do not have a college diploma, four-year diploma. These are people who are truck drivers and who are nail technicians and nurses’ aides.”
None
Instagram posts
stated on August 24, 2022 in an Instagram post:
Image claims to show a list of "Republican members of Congress whose PPP loans were forgiven."
None
League of Conservation Voters
stated on August 13, 2022 in Twitter:
Says Republican U.S. Sen. Ron Johnson “called climate change ‘bull----’ during a record heatwave” and “raked in over $700k in fossil fuel cash.”
None
Mark Pocan
stated on October 3, 2021 in Twitter:
“The annual Pentagon budget hovers over $750 billion annually, twice that of the much debated Build Back Better Act that costs about $350 billion annually as currently proposed.”
None
Mandela Barnes
stated on August 16, 2021 in Twitter:
In Afghanistan, “over 100 billion dollars spent on military contracts.